In [1]:
import json
import pdfkit
import requests
import time
import pandas as pd

In [2]:
base_url = "https://web.archive.org/web/"

# function to go ahead and construct the urls we'll need here
def ret_url(x):
    specific_url = base_url + str(x["timestamp"]) + "/" + x["original"]
    
    return(specific_url)

In [29]:
failure_list = []

# function to print to pdfs the webpages we're choosing here
def print_url(x):
    start = time.time()
    # setting the filename
    file_name = str(x["timestamp"]) +"-" + x["original"] +".pdf"
    
    # removing characters that aren't allowed in a file name
    file_name = file_name.replace(":","-")
    file_name = file_name.replace("/","-")
    
    # checking if the file was originally a pdf or doc; if yes, downloading it
    if ".pdf" in x["original"] or ".doc" in x["original"]:
        # correcting for the issue with the extra .pdf
        file_name = file_name[:len(file_name)-3]
               
        response = requests.get(x["storage_url"])
        
        with open(file_name, "wb") as f:
            f.write(response.content)
    # if not pdf or doc, we're fine to go ahead and just download as pdf w/pdfkit    
    else:
        # trying to download it twice before we declare we failed (odd errors occur afterall)
        try: 
            pdfkit.from_url(x["storage_url"], file_name)
            
        except:
            try:
                pdfkit.from_url(x["storage_url"], file_name)
                
            except:            
                print("issue printing to pdf with:", x["storage_url"])
                failure_list.append(x["storage_url"])
    delay = time.time() - start
    
    # setting a delay based off the time it takes to execute
    time.sleep(2 * delay)


Retrieving the JSON object of URLS

In [25]:
%%time
# starting url
start_url = "https://web.archive.org/cdx/search/cdx?url=http://www.chinese-embassy.org.uk/eng/PressandMedia/&matchType=prefix&output=json&limit=1000&showResumeKey=true&filter=statuscode:200&collapse=digest" 

# using requests to pull the json webpage
r = requests.get(start_url)

# setting the scrape response to the json from the request
scrape_response = r.json()

# as long as the last element is a resume key
while len(scrape_response[-1]) == 1:
    # popping the last item (resume key) & second to last item (blank list)
    resume_key = scrape_response.pop(-1)[0]
    if len(scrape_response[-1]) == 0:
        scrape_response.pop(-1)
        
    # updating the wayback url
    wayback_url = start_url + "&ResumeKey=" + resume_key   
    
    # pulling the resumed query
    
    # error is on here obviously
    r = requests.get(wayback_url).json()
    
    # extending the list of responses
    scrape_response.extend(r)
    
    # sleeping 5 seconds each time to be nice to the server
    time.sleep(5)

Wall time: 598 ms


In [26]:
# converting the cdx response to a df
scrape_df = pd.DataFrame(scrape_response[1:], columns=scrape_response[0])

In [27]:
# removing the duplicates
scrape_df.drop_duplicates(subset="original", inplace=True)

In [28]:
# assigning the storage url
scrape_df["storage_url"] = scrape_df.apply(ret_url, axis=1)

In [ ]:
%%time
# actually downloading all the urls
scrape_df.apply(print_url, axis=1)